In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import numpy as np
import re
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


In [76]:
zomatoDF = pd.read_csv("D:/ISB/FP2/Sandipto-20210914T055751Z-001/Sandipto/datasets/zomato.csv")
zomatoDF = zomatoDF.drop(['url','name','address','phone','menu_item','location'], axis=1)
zomatoDF

,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,cost,listed_in_type,listed_in_city
0,Yes,Yes,4.1,775,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,Buffet,Banashankari
1,Yes,No,4.1,787,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,Buffet,Banashankari
2,Yes,No,3.8,918,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,Buffet,Banashankari
3,No,No,3.7,88,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,Buffet,Banashankari
4,No,No,3.8,166,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...
41660,No,No,3.7,34,"Casual Dining, Bar",NaN,"North Indian, Continental",800.0,Pubs and bars,Whitefield
41661,No,No,2.5,81,"Casual Dining, Bar","Biryani, Andhra Meal","Andhra, South Indian, Chinese, North Indian",800.0,Pubs and bars,Whitefield
41662,No,No,3.6,27,Bar,NaN,Continental,1500.0,Pubs and bars,Whitefield
41663,No,Yes,4.3,236,Bar,"Cocktails, Pizza, Buttermilk",Finger Food,2500.0,Pubs and bars,Whitefield


In [77]:
zomatoDF['dish_liked'] = zomatoDF['dish_liked'].fillna('None')

In [78]:
zomatoDF.isnull().sum()

online_order        0
book_table          0
rate                0
votes               0
rest_type         149
dish_liked          0
cuisines           11
cost              247
listed_in_type      0
listed_in_city      0
dtype: int64

In [7]:
zomatoDF['listed_in_city'].unique()

array(['Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur',
       'Brigade Road', 'Brookefield', 'BTM', 'Church Street',
       'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar',
       'Jayanagar', 'JP Nagar', 'Kalyan Nagar', 'Kammanahalli',
       'Koramangala 4th Block', 'Koramangala 5th Block',
       'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road',
       'Malleshwaram', 'Marathahalli', 'MG Road', 'New BEL Road',
       'Old Airport Road', 'Rajajinagar', 'Residency Road',
       'Sarjapur Road', 'Whitefield'], dtype=object)

In [8]:
zomatoDF['rest_type'].unique()

array(['Casual Dining', 'Cafe, Casual Dining', 'Quick Bites',
       'Casual Dining, Cafe', 'Cafe', 'Quick Bites, Cafe',
       'Cafe, Quick Bites', 'Delivery', 'Mess', 'Dessert Parlor',
       'Bakery, Dessert Parlor', 'Pub', 'Bakery', 'Takeaway, Delivery',
       'Fine Dining', 'Beverage Shop', 'Sweet Shop', 'Bar',
       'Dessert Parlor, Sweet Shop', 'Bakery, Quick Bites',
       'Sweet Shop, Quick Bites', 'Kiosk', 'Food Truck',
       'Quick Bites, Dessert Parlor', 'Beverage Shop, Quick Bites',
       'Beverage Shop, Dessert Parlor', 'Takeaway', 'Pub, Casual Dining',
       'Casual Dining, Bar', 'Dessert Parlor, Beverage Shop',
       'Quick Bites, Bakery', 'Microbrewery, Casual Dining', 'Lounge',
       'Bar, Casual Dining', 'Food Court', 'Cafe, Bakery', nan, 'Dhaba',
       'Quick Bites, Sweet Shop', 'Microbrewery',
       'Food Court, Quick Bites', 'Quick Bites, Beverage Shop',
       'Pub, Bar', 'Casual Dining, Pub', 'Lounge, Bar',
       'Dessert Parlor, Quick Bites', 'Food Co

In [79]:
ohe_cols = ['online_order', 
           'book_table', 
           'listed_in_city', 
           'listed_in_type'
          ]
cv_cols = ['cuisines', 
           'rest_type',
           'dish_liked'
          ]
numeric_cols = ['cost']
y = ['rate']
df_rearranged = zomatoDF[ohe_cols+cv_cols+numeric_cols+y]
df_rearranged.head()

,online_order,book_table,listed_in_city,listed_in_type,cuisines,rest_type,dish_liked,cost,rate
0,Yes,Yes,Banashankari,Buffet,"North Indian, Mughlai, Chinese",Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",800.0,4.1
1,Yes,No,Banashankari,Buffet,"Chinese, North Indian, Thai",Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",800.0,4.1
2,Yes,No,Banashankari,Buffet,"Cafe, Mexican, Italian","Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...",800.0,3.8
3,No,No,Banashankari,Buffet,"South Indian, North Indian",Quick Bites,Masala Dosa,300.0,3.7
4,No,No,Banashankari,Buffet,"North Indian, Rajasthani",Casual Dining,"Panipuri, Gol Gappe",600.0,3.8


In [80]:
train_df, test_df = train_test_split(df_rearranged, test_size=0.3)
train_df

,online_order,book_table,listed_in_city,listed_in_type,cuisines,rest_type,dish_liked,cost,rate
38448,Yes,No,Residency Road,Delivery,"Cafe, Fast Food",Cafe,None,350.0,3.6
22200,Yes,No,Koramangala 4th Block,Delivery,Street Food,Quick Bites,None,100.0,3.5
3362,Yes,Yes,Bellandur,Delivery,"American, Asian, Continental, North Indian, So...","Casual Dining, Pub","Cocktails, Noodles, Chips, Shephards Pie, Lamb...",1200.0,4.1
1378,Yes,No,Bannerghatta Road,Delivery,"Seafood, Salad, North Indian, Biryani, Chinese","Takeaway, Delivery",None,500.0,3.2
38995,No,Yes,Residency Road,Dine-out,"Italian, North Indian",Fine Dining,"Salads, Brownie, Cheesecake, Salad, Biryani, D...",3000.0,4.1
...,...,...,...,...,...,...,...,...,...
34140,Yes,No,MG Road,Delivery,"Cafe, Continental, Momos, Italian, North Indian","Cafe, Casual Dining","Momos, Wedges, Chai, Veg Club Sandwich, Masala...",500.0,4.1
13927,Yes,No,Indiranagar,Cafes,"Cafe, Italian, American","Cafe, Casual Dining","Pizza, Cheesy Fries, Tiramisu, Ravioli Pasta, ...",1000.0,4.3
919,Yes,No,Bannerghatta Road,Delivery,"North Indian, Chinese, Fast Food, Rolls, Juices",Quick Bites,"Shawarma, Chicken Grill, Tandoori Chicken, But...",400.0,2.5
38467,Yes,No,Residency Road,Delivery,"Chinese, North Indian",Quick Bites,None,450.0,3.6


In [81]:
X_train = np.empty(shape=(len(train_df),0))
X_test = np.empty(shape=(len(test_df),0))
dict_of_transformers = {}

In [82]:
transformer_type = 'one_hot_encoder'
dict_of_transformers[transformer_type] = OneHotEncoder(drop='first', sparse=False)
X = dict_of_transformers[transformer_type].fit_transform(train_df[ohe_cols])
print('Categories found:: ', dict_of_transformers[transformer_type].get_feature_names())
display('Original column', train_df[ohe_cols].head(10))
display('Transformed df:::',pd.DataFrame(data=X, columns=dict_of_transformers[transformer_type].get_feature_names()).head(10))

Categories found::  ['x0_Yes' 'x1_Yes' 'x2_Banashankari' 'x2_Bannerghatta Road'
 'x2_Basavanagudi' 'x2_Bellandur' 'x2_Brigade Road' 'x2_Brookefield'
 'x2_Church Street' 'x2_Electronic City' 'x2_Frazer Town' 'x2_HSR'
 'x2_Indiranagar' 'x2_JP Nagar' 'x2_Jayanagar' 'x2_Kalyan Nagar'
 'x2_Kammanahalli' 'x2_Koramangala 4th Block' 'x2_Koramangala 5th Block'
 'x2_Koramangala 6th Block' 'x2_Koramangala 7th Block' 'x2_Lavelle Road'
 'x2_MG Road' 'x2_Malleshwaram' 'x2_Marathahalli' 'x2_New BEL Road'
 'x2_Old Airport Road' 'x2_Rajajinagar' 'x2_Residency Road'
 'x2_Sarjapur Road' 'x2_Whitefield' 'x3_Cafes' 'x3_Delivery' 'x3_Desserts'
 'x3_Dine-out' 'x3_Drinks & nightlife' 'x3_Pubs and bars']


'Original column'

,online_order,book_table,listed_in_city,listed_in_type
38448,Yes,No,Residency Road,Delivery
22200,Yes,No,Koramangala 4th Block,Delivery
3362,Yes,Yes,Bellandur,Delivery
1378,Yes,No,Bannerghatta Road,Delivery
38995,No,Yes,Residency Road,Dine-out
17397,Yes,No,JP Nagar,Cafes
41176,Yes,No,Whitefield,Dine-out
2371,Yes,Yes,Basavanagudi,Delivery
16218,Yes,No,Jayanagar,Delivery
2739,No,No,Basavanagudi,Dine-out


'Transformed df:::'

,x0_Yes,x1_Yes,x2_Banashankari,x2_Bannerghatta Road,x2_Basavanagudi,x2_Bellandur,x2_Brigade Road,x2_Brookefield,x2_Church Street,x2_Electronic City,...,x2_Rajajinagar,x2_Residency Road,x2_Sarjapur Road,x2_Whitefield,x3_Cafes,x3_Delivery,x3_Desserts,x3_Dine-out,x3_Drinks & nightlife,x3_Pubs and bars
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [83]:
X_train = np.hstack((X_train,X))
X_test = np.hstack((X_test,dict_of_transformers[transformer_type].transform(test_df[ohe_cols])))
X_train.shape, X_test.shape

((29165, 37), (12500, 37))

In [84]:
for col in cv_cols:
    print('\n******', col,'*****')
    # fill na
    train_df[col] = train_df[col].fillna('unk')
    test_df[col] = test_df[col].fillna('unk')
    regex = '\, '
    dict_of_transformers[col] = CountVectorizer(tokenizer=lambda text: re.split(regex,text), binary=True, min_df=10)
    X = dict_of_transformers[col].fit_transform(train_df[col])
    X = X.toarray()
    print('# of Categories found:: ', len(dict_of_transformers[col].get_feature_names()))
    display('Original column', train_df[col].head(10))
    display('Transformed df:::',pd.DataFrame(data=X, columns=dict_of_transformers[col].get_feature_names()).head(10))
    
    X_train = np.hstack((X_train,X))
    X_test_temp = dict_of_transformers[col].transform(test_df[col]).toarray()
    X_test = np.hstack((X_test,X_test_temp))
    print('X_train.shape, X_test.shape=',X_train.shape, X_test.shape)


****** cuisines *****
# of Categories found::  84


'Original column'

38448                                      Cafe, Fast Food
22200                                          Street Food
3362     American, Asian, Continental, North Indian, So...
1378        Seafood, Salad, North Indian, Biryani, Chinese
38995                                Italian, North Indian
17397                              Cafe, Burger, Beverages
41176                               Maharashtrian, Seafood
2371      Cafe, Asian, Burger, Continental, Italian, Salad
16218                                         North Indian
2739           Andhra, South Indian, Biryani, North Indian
Name: cuisines, dtype: object

'Transformed df:::'

,afghan,african,american,andhra,arabian,asian,assamese,awadhi,bakery,bar food,...,steak,street food,sushi,tea,tex-mex,thai,tibetan,turkish,vietnamese,wraps
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


...

'Original column'

38448                                                 None
22200                                                 None
3362     Cocktails, Noodles, Chips, Shephards Pie, Lamb...
1378                                                  None
38995    Salads, Brownie, Cheesecake, Salad, Biryani, D...
17397    Burgers, Mojito, Lamb Burger, French Fries, Ho...
41176    Misal Pav, Mutton Fry, Malwani Food, Mutton Th...
2371     Burgers, Egg Salad, Pasta, Pancakes, Lasagne, ...
16218    Paratha, Chicken Kebab, Biryani, Lassi, Butter...
2739                                                  None
Name: dish_liked, dtype: object

'Transformed df:::'

,aalo bhaja,aam panna,aamras,akki rottis,al faham chicken,almond fudge,almond soup,aloo palya,aloo paratha,aloo posto,...,white pasta,white sauce pasta,whole meat shawarma,whopper burger,wine,wings,wonton soup,wood fire pizza,zafrani mutton biryani,zinger burger
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


X_train.shape, X_test.shape= (29165, 1247) (12500, 1247)


In [85]:
# Impute Cost
X_train = np.hstack((X_train,train_df.cost.values.reshape(-1,1)))
X_test = np.hstack((X_test,test_df.cost.values.reshape(-1,1)))

dict_of_transformers['imputer'] = KNNImputer(n_neighbors=5)
X_train = dict_of_transformers['imputer'].fit_transform(X_train)
X_test = dict_of_transformers['imputer'].transform(X_test)

In [86]:
dict_of_transformers['cost_scaler'] = StandardScaler()
dict_of_transformers['cost_scaler'].fit(X_train[:,-1].reshape(-1,1))
X_train[:,-1] = dict_of_transformers['cost_scaler'].transform(X_train[:,-1].reshape(-1,1)).reshape(-1,)
X_test[:,-1] = dict_of_transformers['cost_scaler'].transform(X_test[:,-1].reshape(-1,1)).reshape(-1,)

In [87]:
input_layer = tf.keras.Input(shape=(X_train.shape[1],), name='input_layer', dtype='float32')
h1 = tf.keras.layers.Dense(16, activation='relu', name='h1')(input_layer)
outputs = tf.keras.layers.Dense(1, activation='relu', name='output')(h1)

model = tf.keras.Model(inputs=input_layer,outputs=outputs)

In [88]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer,loss=loss)

In [89]:
epochs = 50
callbacks = [tf.keras.callbacks.EarlyStopping(patience=10,verbose=1, restore_best_weights=True)]
history = model.fit(x=X_train,y=train_df.rate.values, 
                  batch_size=32,
                  validation_data=(X_test,test_df.rate.values),
                  validation_batch_size=32,
                  epochs=epochs,
                  callbacks=callbacks
                 )

Epoch 1/50
912/912 [==============================] - 3s 3ms/step - loss: 0.2687 - val_loss: 0.1252
Epoch 2/50
912/912 [==============================] - 2s 2ms/step - loss: 0.1064 - val_loss: 0.1014
Epoch 3/50
912/912 [==============================] - 2s 2ms/step - loss: 0.0921 - val_loss: 0.0964
Epoch 4/50
912/912 [==============================] - 2s 2ms/step - loss: 0.0848 - val_loss: 0.0900
Epoch 5/50
912/912 [==============================] - 2s 2ms/step - loss: 0.0813 - val_loss: 0.0812
Epoch 6/50
912/912 [==============================] - 3s 3ms/step - loss: 0.0760 - val_loss: 0.0779
Epoch 7/50
912/912 [==============================] - 2s 3ms/step - loss: 0.0727 - val_loss: 0.0746
Epoch 8/50
912/912 [==============================] - 2s 3ms/step - loss: 0.0688 - val_loss: 0.0726
Epoch 9/50
912/912 [==============================] - 2s 3ms/step - loss: 0.0668 - val_loss: 0.0720
Epoch 10/50
912/912 [==============================] - 2s 3ms/step - loss: 0.0642 - val_loss: 0.0677

In [91]:
X_train.shape

(29165, 1248)

In [56]:
history

In [90]:
from sklearn.metrics import r2_score
y_pred = model.predict(X_test).reshape(-1,)
y_pred = np.round(y_pred,decimals=1)
y_test = test_df.rate.values.reshape(-1,)
print('Pseudo R2: ', r2_score(y_true=y_test,y_pred=y_pred))

Pseudo R2:  0.7120979280678534


In [59]:
kf = KFold(n_splits=2)

In [61]:
kfold = KFold(n_splits=5)
cvscores = []
X = X_train
Y =  train_df.rate.values
for train, test in kfold.split(X, Y):
  # create model
    input_layer = tf.keras.Input(shape=(X_train.shape[1],), name='input_layer', dtype='float32')
    h1 = tf.keras.layers.Dense(16, activation='relu', name='h1')(input_layer)
    outputs = tf.keras.layers.Dense(1, activation='relu', name='output')(h1)
    model = tf.keras.Model(inputs=input_layer,outputs=outputs)
    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    loss = tf.keras.losses.MeanSquaredError()
    model.compile(optimizer=optimizer,loss=loss)
    # Fit the model
    epochs = 50
    callbacks = [tf.keras.callbacks.EarlyStopping(patience=10,verbose=1, restore_best_weights=True)]
    history = model.fit(x=X_train,y=train_df.rate.values, 
                      batch_size=32,
                      validation_data=(X_test,test_df.rate.values),
                      validation_batch_size=32,
                      epochs=epochs,
                      callbacks=callbacks
                     )
    # evaluate the model
    
    y_pred = model.predict(X_test).reshape(-1,)
    y_pred = np.round(y_pred,decimals=1)
    y_test = test_df.rate.values.reshape(-1,)
    r2 = r2_score(y_true=y_test,y_pred=y_pred)
    print('Pseudo R2: ', r2)
    cvscores.append(r2 * 100)
    #scores = model.evaluate(X[test], Y[test], verbose=0)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    #cvscores.append(scores[1] * 100)
    


Epoch 1/50
912/912 [==============================] - 2s 2ms/step - loss: 0.2723 - val_loss: 0.1477
Epoch 2/50
912/912 [==============================] - 2s 2ms/step - loss: 0.1445 - val_loss: 0.1352
Epoch 3/50
912/912 [==============================] - 1s 2ms/step - loss: 0.1368 - val_loss: 0.1341
Epoch 4/50
912/912 [==============================] - 1s 2ms/step - loss: 0.1314 - val_loss: 0.1249
Epoch 5/50
912/912 [==============================] - 2s 2ms/step - loss: 0.1272 - val_loss: 0.1250
Epoch 6/50
912/912 [==============================] - 2s 2ms/step - loss: 0.1248 - val_loss: 0.1383
Epoch 7/50
912/912 [==============================] - 2s 2ms/step - loss: 0.1221 - val_loss: 0.1349
Epoch 8/50
912/912 [==============================] - 2s 2ms/step - loss: 0.1215 - val_loss: 0.1311
Epoch 9/50
912/912 [==============================] - 2s 2ms/step - loss: 0.1190 - val_loss: 0.1214
Epoch 10/50
912/912 [==============================] - 2s 2ms/step - loss: 0.1178 - val_loss: 0.1164

NameError: name 'numpy' is not defined

In [62]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

44.39% (+/- 0.97%)


In [92]:
X = X_train
Y =  train_df.rate.values


# Random Forest Model
rf = RandomForestRegressor(n_estimators = 800, random_state = 42, max_features='log2', min_samples_split=2, min_samples_leaf=1, max_depth=20, bootstrap=False)
rf.fit(X, Y) #training the algorithm
y_pred = rf.predict(X_test)
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3) 
print('RMSE -> ', rmse)
r2 = round(r2_score(y_test, y_pred) * 100,2) 
print("R2 -> ",r2)
r2Adj = round((1 - (1-rf.score(X_test,y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)) * 100,2) 
print("R2 Adjusted -> ",r2Adj) 
mae = round(mean_absolute_error(y_test, y_pred),3)
print("MAE -> ",mae)

RMSE ->  0.349
R2 ->  37.76
R2 Adjusted ->  30.86
MAE ->  0.269


In [93]:
# XGBoost Model
xgb = XGBRegressor(n_estimators=1000, max_depth=20, eta=0.1, subsample=0.9, colsample_bytree=0.8, reg_lambda=1.2, reg_alpha=1.1)
xgb.fit(X, Y) #training the algorithm
y_pred = xgb.predict(X_test)
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3) 
print('RMSE -> ', rmse)
r2 = round(r2_score(y_test, y_pred) * 100,2) 
print("R2 -> ",r2)
r2Adj = round((1 - (1-xgb.score(X_test,y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)) * 100,2) 
print("R2 Adjusted -> ",r2Adj) 
mae = round(mean_absolute_error(y_test, y_pred),3)
print("MAE -> ",mae)

RMSE ->  0.174
R2 ->  84.42
R2 Adjusted ->  82.69
MAE ->  0.107
